You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

# Summary

Access to large, high quality data is essential for researchers to understand disease and precision medicine pathways, especially in cancer. However HIPAA constraints make sharing medical images outside an individual institution a complex process. [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a public service funded by the National Cancer Institute which addresses this challenge by providing hosting and de-identification services to take major burdens of data sharing off researchers.

**This notebook is focused on basic use cases for leveraging TCIA's REST APIs to query and download data.**  If you're interested in additional TCIA notebooks and coding examples check out https://github.com/kirbyju/TCIA_Notebooks.

# 1 Learn about available Collections on the TCIA website

[Browsing Collections](https://www.cancerimagingarchive.net/collections) and [Analysis Results](https://www.cancerimagingarchive.net/tcia-analysis-results/) datasets on TCIA are the easiest ways to become familiar with what is available.  These pages will help you quickly identify datasets of interest, find valuable supporting data that are not available via our APIs (e.g. clinical spreadsheets, non-DICOM segmentation data), and answer most common questions you might have about the datasets.  

# 2 REST API Overview
TCIA uses software called NBIA to manage DICOM data.  The NBIA REST APIs include:
1. [NBIA Search REST APIs](https://wiki.cancerimagingarchive.net/x/fILTB) that allow you to perform basic queries and download data from **public** collections. This API does not require a TCIA account.
2. [NBIA Search with Authentication REST APIs](https://wiki.cancerimagingarchive.net/x/X4ATBg) that allow you to perform basic queries and download data from **public and limited-access** collections. This API requires a TCIA account for creation of authentication tokens.
3. [NBIA Advanced REST APIs](https://wiki.cancerimagingarchive.net/x/YoATBg) that allow access to **public and limited-access** collections, but provide query endpoints mostly geared towards developers seeking to integrate searching and downloading TCIA data into web and desktop applications.  This API requires a TCIA account for creation of authentication tokens.

As of v2.3 **tcia_utils** you don't have to worry about this complexity.  Each function automatically calls the proper API, and if credentials aren't specified you are logged in via a guest account to view fully public data.  See section 4.5 of this notebook to learn how to download "limited access" datasets you have received permission to use.

# 3 Setup

The following cells install and import the **nbia** from [**tcia_utils**](https://pypi.org/project/tcia-utils/), which contains a variety of useful functions for accessing TCIA via Python, and also import **dicomViewer** from [**simpleDicomViewer**](https://pypi.org/project/simpleDicomViewer/) for visualizing DICOM series.

**tcia_utils** contains a **downloadSeries()** function that has multiple options for specifying the seriesUids you'd like to download.  By default, the function expects JSON data containing "SeriesInstanceUID" elements, which can be generated using any of the series-related queries such as **getSeries()** or **getCart()**.  However, if you have a series UID list from some other source, you can set **input_type = "list"** to pass a python list of one or more series UIDs instead of JSON. You can also set **input_type = "manifest"** to provide a path to a **.TCIA** manifest file.

Data are saved to a **tciaDownload** folder in your current working directory by default, but you can use the **path** parameter to change this to a different directory.  Data are unzipped into a folder named after the Series Instance UID, unless `as_zip` is set to True.  In this case, it skips the unzipping steps and names the zip file after the Series Instance UID.  

There is an optional **format** parameter that can be used to return metadata about what was downloaded.  It can be set to **df** to return dataframe or **csv** to save a spreadsheet. There's also a **csv_filename** parameter if you want to set a specific file name.

You can specify **number = n** to tell the function to only download the first **n** scans of your seriesUids.  Remove this parameter in the examples below to download the full dataset.

The **api_url** parameter can be omitted in most cases.  However, it must be set to **api_url = "nlst"** to access the [National Lung Screening Trial (NLST)](https://doi.org/10.7937/TCIA.HMQ8-J677) collection.  

Last but not least, there is some logic built in to detect whether you've already downloaded a series.  If a directory named after the seriesUid already exists the function will assume it's already been downloaded and skip that series.

In [1]:
import sys

# install tcia utils
!{sys.executable} -m pip install --upgrade -q tcia_utils

# install simpleDicomViewer
!{sys.executable} -m pip install --upgrade -q simpleDicomViewer

In [2]:
import requests
import pandas as pd
from tcia_utils import nbia
from simpleDicomViewer import dicomViewer

# set logging level to INFO in Google Colab (not necessary in Jupyter)
if 'google.colab' in sys.modules:
  import logging

  for handler in logging.root.handlers[:]:
      logging.root.removeHandler(handler)

  # Set handler with level = info
  logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                      level=logging.INFO)

  print("Google Colab Logging set to INFO")

Google Colab Logging set to INFO


# 4 Download Examples

In this section we'll cover downloading data via the REST API for the following use cases:

1.   Download a full TCIA collection
2.   Download custom results of an API query
3.   Download a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" that was created via https://nbia.cancerimagingarchive.net/
4.   Download data from a TCIA manifest file
5.   Download data from a **restricted** collection that requires logging in with your account
6.   Download individual images from a series

## 4.1 Download a full collection

You can [Browse Collections](https://www.cancerimagingarchive.net/collections) on our website to figure out what you might want to download, but you can also get a list of available collections via the API as shown below.

In [3]:
# get list of available collections as JSON
nbia.getCollections()

2025-05-12 06:48:26,099:INFO:Success - Token saved to global api_call_headers variable and expires at 2025-05-12 08:48:26.099497
2025-05-12 06:48:26,100:INFO:Accessing public data anonymously. To access restricted data use nbia.getToken() with your credentials.
2025-05-12 06:48:26,102:INFO:Calling getCollectionValues with parameters {}


[{'Collection': '4D-Lung'},
 {'Collection': 'ACRIN-6698'},
 {'Collection': 'ACRIN-Contralateral-Breast-MR'},
 {'Collection': 'ACRIN-FLT-Breast'},
 {'Collection': 'ACRIN-NSCLC-FDG-PET'},
 {'Collection': 'Adrenal-ACC-Ki67-Seg'},
 {'Collection': 'Advanced-MRI-Breast-Lesions'},
 {'Collection': 'Anti-PD-1_Lung'},
 {'Collection': 'B-mode-and-CEUS-Liver'},
 {'Collection': 'BREAST-DIAGNOSIS'},
 {'Collection': 'Breast-Cancer-Screening-DBT'},
 {'Collection': 'Breast-MRI-NACT-Pilot'},
 {'Collection': 'C4KC-KiTS'},
 {'Collection': 'CBIS-DDSM'},
 {'Collection': 'CC-Radiomics-Phantom'},
 {'Collection': 'CC-Radiomics-Phantom-2'},
 {'Collection': 'CC-Radiomics-Phantom-3'},
 {'Collection': 'CC-Tumor-Heterogeneity'},
 {'Collection': 'CMB-AML'},
 {'Collection': 'CMB-BRCA'},
 {'Collection': 'CMB-CRC'},
 {'Collection': 'CMB-GEC'},
 {'Collection': 'CMB-LCA'},
 {'Collection': 'CMB-MEL'},
 {'Collection': 'CMB-MML'},
 {'Collection': 'CMB-OV'},
 {'Collection': 'CMB-PCA'},
 {'Collection': 'CMMD'},
 {'Collection'

Let's say that we're interested in downloading the entire **Soft-tissue-Sarcoma** collection.  First we need to get a list of all Series Instance UIDs in that collection.  We can use **nbia.getSeries()** to return JSON metadata about all series (scans) in this collection.

In [4]:
data = nbia.getSeries(collection = "Soft-tissue-Sarcoma")
print(data)

2025-05-12 06:48:33,327:INFO:Calling getSeries with parameters {'Collection': 'Soft-tissue-Sarcoma'}


[{'SeriesInstanceUID': '1.3.6.1.4.1.14519.5.2.1.5168.1900.104193299251798317056218297018', 'StudyInstanceUID': '1.3.6.1.4.1.14519.5.2.1.5168.1900.154535988064062152660648619556', 'Modality': 'MR', 'SeriesDate': '2003-12-12 00:00:00.0', 'SeriesDescription': '2. AXIAL T1 BOTH LEGS - RESEARCH', 'BodyPartExamined': 'EXTREMITY', 'SeriesNumber': 2, 'Collection': 'Soft-tissue-Sarcoma', 'PatientID': 'STS_010', 'Manufacturer': 'GE MEDICAL SYSTEMS', 'ManufacturerModelName': 'GENESIS_SIGNA', 'SoftwareVersions': '09', 'ImageCount': 48, 'TimeStamp': '2015-05-27 17:12:21.0', 'LicenseName': 'Creative Commons Attribution 3.0 Unported License', 'LicenseURI': 'http://creativecommons.org/licenses/by/3.0/', 'CollectionURI': 'https://doi.org/10.7937/K9/TCIA.2015.7GO2GSKS', 'FileSize': 25273786, 'DateReleased': '2015-05-27 17:12:21.0', 'StudyDesc': 'MRI LT LEG +C', 'StudyDate': '2003-12-12 00:00:00.0'}, {'SeriesInstanceUID': '1.3.6.1.4.1.14519.5.2.1.5168.1900.614409775072665417753627581100', 'StudyInstanceU

We can then pass that to the our download functions and view/save the metadata for what was downloaded.  We'll leverage the **number** parameter here to just grab the first 2 scans as a test.  You can remove this parameter if you want to download the full collection.

In [5]:
nbia.downloadSeries(data, number = 2)

2025-05-12 06:48:39,858:INFO:Downloading 2 out of 612 Series Instance UIDs (scans).
2025-05-12 06:48:39,859:INFO:Directory 'tciaDownload' already exists.
2025-05-12 06:48:39,860:WARNING:Series 1.3.6.1.4.1.14519.5.2.1.5168.1900.104193299251798317056218297018 already downloaded and unzipped.
2025-05-12 06:48:39,861:WARNING:Series 1.3.6.1.4.1.14519.5.2.1.5168.1900.614409775072665417753627581100 already downloaded and unzipped.
2025-05-12 06:48:39,863:WARNING:Series 1.3.6.1.4.1.14519.5.2.1.5168.1900.125236013959219285601516346712 already downloaded and unzipped.
2025-05-12 06:48:39,864:WARNING:Series 1.3.6.1.4.1.14519.5.2.1.5168.1900.216398125896037785163292266798 already downloaded and unzipped.
2025-05-12 06:48:39,865:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.5168.1900.246858556033491753396338124091
2025-05-12 06:48:42,881:INFO:Downloading... https://services.cancerimagingarchive.

Take a second to go look at your **tciaDownload** folder to view the data.  Note that each series is saved in a directory named by its Series Instance UID.

You can learn more about various ways to visualize and analyze your data in the other notebooks at https://github.com/kirbyju/TCIA_Notebooks. However, let's use **dicomViewer.viewSeries()** from the **simpleDicomViewer** package to get a quick look at one of the series we've downloaded. You can change the Series UID in the code below to view other scans you've downloaded.


In [6]:
seriesUid = "1.3.6.1.4.1.14519.5.2.1.5168.1900.104193299251798317056218297018"
dicomViewer.viewSeries(f"tciaDownload/{seriesUid}")

interactive(children=(IntSlider(value=23, description='x', max=47), Output()), _dom_classes=('widget-interact'…

## 4.2 Download custom API query
The REST API allows for a variety of different query options as demonstrated in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Queries.ipynb).  

### 4.2.1 Custom downloads with getSeries()

For this use case, let's assume that you are only interested in the MR scans from the [TCGA-BRCA](https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP) collection that were acquired on Siemens scanners.

In [11]:
# getSeries with query parameters
data = nbia.getSeries(collection = "TCGA-BRCA",
                      modality = "MR",
                      manufacturer = "SIEMENS")

print(len(data), 'Series returned')

2025-05-12 06:30:29,514:INFO:Calling getSeries with parameters {'Collection': 'TCGA-BRCA', 'Modality': 'MR', 'Manufacturer': 'SIEMENS'}


353 Series returned


Once again, let's pass those Series Instance UIDs to our download function.  This time we'll also set **format = "df"** to return a dataframe about what we downloaded.

In [15]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 2, format = "df")
display(df)

2025-05-12 06:32:10,106:INFO:Downloading 2 out of 353 Series Instance UIDs (scans).
2025-05-12 06:32:10,113:INFO:Directory 'tciaDownload' already exists.
2025-05-12 06:32:10,114:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.1869.4002.209741730939031619108046227579
2025-05-12 06:32:12,898:INFO:Downloading... https://services.cancerimagingarchive.net/nbia-api/services/v2/getImage?NewFileNames=Yes&SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.1869.4002.505294946139745851309925972851
2025-05-12 06:32:14,500:INFO:Downloaded 2 out of 2 Series Instance UIDs (scans).
0 failed to download.
0 previously downloaded.


,Series UID,Collection,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,...,License URL,Annotation Size,Date Released,Series Date,Protocol Name,Body Part Examined,Annotations Flag,Manufacturer Model Name,Software Versions,TimeStamp
0,1.3.6.1.4.1.14519.5.2.1.1869.4002.209741730939...,TCGA-BRCA,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,TCGA-EW-A1IY,1.3.6.1.4.1.14519.5.2.1.1869.4002.323075922339...,BreastBilateral Tumor,06-20-2001,T1 se ax bilat,SIEMENS,MR,...,http://creativecommons.org/licenses/by/3.0/,0,Mon Nov 04 11:45:52 UTC 2013,Wed Jun 20 00:00:00 UTC 2001,T1 se ax bilat,BREAST,false,Avanto,syngo MR B15,Mon Nov 04 11:45:52 UTC 2013
1,1.3.6.1.4.1.14519.5.2.1.1869.4002.505294946139...,TCGA-BRCA,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,TCGA-EW-A1IY,1.3.6.1.4.1.14519.5.2.1.1869.4002.323075922339...,BreastBilateral Tumor,06-20-2001,T2 fse ax bilat,SIEMENS,MR,...,http://creativecommons.org/licenses/by/3.0/,0,Mon Nov 04 11:45:59 UTC 2013,Wed Jun 20 00:00:00 UTC 2001,T2 fse ax bilat,BREAST,false,Avanto,syngo MR B15,Mon Nov 04 11:45:59 UTC 2013


Another common use case may be that you'd like to review the results from **getSeries()** and do some further filtering.  This time we'll add **format = "df"** to save the output to a dataframe.  After removing unwanted scans from the dataframe, we'll pass the remaining Series Instance UIDs to **downloadSeries()**.  For example, let's say that you only wanted to download **T2 MR** series instead of any MR.

In [16]:
# getSeries with query parameters
df = nbia.getSeries(collection = "TCGA-BRCA",
               modality = "MR",
               manufacturer = "SIEMENS", format = "df")

2025-05-12 06:33:42,244:INFO:Calling getSeries with parameters {'Collection': 'TCGA-BRCA', 'Modality': 'MR', 'Manufacturer': 'SIEMENS'}


Let's drop scans that don't contain **t2** in either the **ProtocolName** or **SeriesDescription** fields.  Note that we're down to 21 scans now instead of 353.

In [17]:
# convert the columns to lowercase to catch both upper/lower case with filter for 't2'
filtered_df = df[(df['ProtocolName'].str.lower().str.contains('t2')) |
                 (df['SeriesDescription'].str.lower().str.contains('t2'))]

display(filtered_df)

,SeriesInstanceUID,StudyInstanceUID,Modality,ProtocolName,SeriesDate,SeriesDescription,BodyPartExamined,SeriesNumber,Collection,PatientID,...,SoftwareVersions,ImageCount,TimeStamp,LicenseName,LicenseURI,CollectionURI,FileSize,DateReleased,StudyDesc,StudyDate
1,1.3.6.1.4.1.14519.5.2.1.1869.4002.505294946139...,1.3.6.1.4.1.14519.5.2.1.1869.4002.323075922339...,MR,T2 fse ax bilat,2001-06-20 00:00:00.0,T2 fse ax bilat,BREAST,5,TCGA-BRCA,TCGA-EW-A1IY,...,syngo MR B15,36,2013-11-04 11:45:59.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,4832782,2013-11-04 11:45:59.0,Breast^Bilateral Tumor,2001-06-20 00:00:00.0
30,1.3.6.1.4.1.14519.5.2.1.1869.4002.199871253483...,1.3.6.1.4.1.14519.5.2.1.1869.4002.108214931724...,MR,T2 fse ax bilat,2001-11-02 00:00:00.0,T2 fse ax bilat,BREAST,4,TCGA-BRCA,TCGA-EW-A1J6,...,syngo MR B15,36,2013-11-04 11:56:38.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,4834576,2013-11-04 11:56:38.0,Breast^Bilateral Tumor,2001-11-02 00:00:00.0
72,1.3.6.1.4.1.14519.5.2.1.9203.4002.320534549323...,1.3.6.1.4.1.14519.5.2.1.9203.4002.598259735076...,MR,AX T2 NON FS,2003-12-13 00:00:00.0,AX T2 NON FS,BREAST,4,TCGA-BRCA,TCGA-AO-A0JF,...,syngo MR B15,78,2012-06-29 17:38:39.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,41084274,2012-06-29 17:38:39.0,NaN,2003-12-13 00:00:00.0
82,1.3.6.1.4.1.14519.5.2.1.1869.4002.112537292906...,1.3.6.1.4.1.14519.5.2.1.1869.4002.894763170975...,MR,TSE T2 AXIALS,2001-08-31 00:00:00.0,TSE T2 AXIALS,BREAST,4,TCGA-BRCA,TCGA-EW-A1OY,...,syngo MR 2004A 4VA25A,80,2013-11-04 12:12:52.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,42177918,2013-11-04 12:12:52.0,BODY^BREAST,2001-08-31 00:00:00.0
91,1.3.6.1.4.1.14519.5.2.1.5382.4002.140243587673...,1.3.6.1.4.1.14519.5.2.1.5382.4002.104582989590...,MR,t2_tirm_cor,2003-05-04 00:00:00.0,t2_tirm_cor,BREAST,2,TCGA-BRCA,TCGA-OL-A5DA,...,syngo MR 2004A 4VA25A,28,2014-12-26 16:27:20.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,3762886,2014-12-26 16:27:20.0,MRI BREAST W/WO + SUB + RECON,2003-05-04 00:00:00.0
92,1.3.6.1.4.1.14519.5.2.1.5382.4002.806935685832...,1.3.6.1.4.1.14519.5.2.1.5382.4002.104582989590...,MR,t2_tse__tra_,2003-05-04 00:00:00.0,t2_tse__tra_,BREAST,5,TCGA-BRCA,TCGA-OL-A5DA,...,syngo MR 2004A 4VA25A,41,2014-12-26 16:27:33.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,21630440,2014-12-26 16:27:33.0,MRI BREAST W/WO + SUB + RECON,2003-05-04 00:00:00.0
97,1.3.6.1.4.1.14519.5.2.1.5382.4002.106017098988...,1.3.6.1.4.1.14519.5.2.1.5382.4002.104582989590...,MR,t2_tirm_tra,2003-05-04 00:00:00.0,t2_tirm_tra,BREAST,7,TCGA-BRCA,TCGA-OL-A5DA,...,syngo MR 2004A 4VA25A,41,2014-12-26 16:29:24.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,5509158,2014-12-26 16:29:24.0,MRI BREAST W/WO + SUB + RECON,2003-05-04 00:00:00.0
107,1.3.6.1.4.1.14519.5.2.1.5382.4002.237274108013...,1.3.6.1.4.1.14519.5.2.1.5382.4002.288973310599...,MR,t2_tirm_cor,2006-10-10 00:00:00.0,t2_tirm_cor,BREAST,3,TCGA-BRCA,TCGA-OL-A66O,...,syngo MR A30 4VA30A,24,2020-04-22 16:11:42.0,Creative Commons Attribution 3.0 Unported License,http://creativecommons.org/licenses/by/3.0/,https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP,12656088,2020-04-22 16:11:42.0,MRI BREAST BIL WO W CONTRAST,2006-10-10 00:00:00.0
108,1.3.6.1.4.1.14519.5.2.1.5382.4002.156252807654...,1.3.6.1.4.1.14519.5.2.1.5382.4002.288973310599...,MR,t2_tirm_tra_5_12_10,2006-10-10 00:00:0

Now we can feed the Series Instance UIDs from this dataframe back to **downloadSeries()**.  Note that since we're not working with the API's default JSON output anymore we need to specify **input_type = "df"** when we call **downloadSeries()**.

Let's also try out the `as_zip` parameter this time just for fun.

In [ ]:
# download the selected series_uids
nbia.downloadSeries(filtered_df, input_type = "df", number = 2, as_zip=True)

### 4.2.2 Custom downloads with  getSimpleSearchWithModalityAndBodyPartPaged()
This function lets you do pretty much everything you can do in the **Simple Search** screen at https://nbia.cancerimagingarchive.net. As as a result, there are a lot of parameters.

```
collections: list[str]   -- The DICOM collections of interest to you
    species: list[str]       -- Filter collections by species. Possible values are 'human', 'mouse', and 'dog'
    modalities: list[str]    -- Filter collections by modality
    modalityAnded: bool      -- If true, only return subjects with all requested modalities, as opposed to any
    minStudies: int          -- The minimum number of studies a collection must have to be included in the results
    manufacturers: list[str] -- Imaging device manufacturers, e.g. SIEMENS
    bodyParts: list[str]     -- Body parts of interest, e.g. CHEST, ABDOMEN
    fromDate: str            -- First cutoff date, in YYYY/MM/DD format. Defaults to 1900/01/01
    toDate: str              -- Second cutoff date, in YYYY/MM/DD format. Defaults to today's date
    patients: list[str]      -- Patients to include in the output
    start: int               -- Starting point of returned subject results. Defaults to 0.
    size: int                -- Number of returned subjects per page. Defaults to 10.
    sortDirection            -- 'ascending' or 'descending'. Defaults to 'ascending'.
    sortField                -- 'subject', 'studies', 'series', or 'collection'. Defaults to 'subject'.
    format: str              -- Defaults to JSON. Can be set to "uids" to return a python list of
                                Series Instance UIDs or "manifest" to save a TCIA manifest file (up to 1,000,000 series).
```

By default it returns paged results in JSON format, but you can also set it to **manifest** or **uids** to return the results in a way that makes it easy to download them with `nbia.downloadSeries()`.  You just need to set the `input_type` parameter of `downloadSeries()` and the `format` parameter of `getSimpleSearchWithModalityAndBodyPartPaged()` accordingly.  For example:

In [ ]:
uids = nbia.getSimpleSearchWithModalityAndBodyPartPaged(bodyParts=["CHEST"], modalities=["CT"], format = "uids")

In [ ]:
# just grab the first series using number = 1 for demonstration purposes
nbia.downloadSeries(uids, input_type = "list", number = 1)

Another common use case might be that you want to download a specific set of Patient IDs.  You can combine this with other query parameters as shown below.  Let's say you only want chest xrays for these two LIDC subjects:

In [ ]:
# set patient ID list
ids = ['LIDC-IDRI-0001', 'LIDC-IDRI-0002']

# submit query
uids = nbia.getSimpleSearchWithModalityAndBodyPartPaged(patients=ids, modalities = ["CR", "DX"], format = "uids")

# download images
nbia.downloadSeries(uids, input_type = "list", format = "df")

### 4.2.3 Download custom NLST API query
Let's show a similar example where we look for a specific modality and manufacturer within the [National Lung Screening Trial (NLST) Collection](https://doi.org/10.7937/TCIA.HMQ8-J677).  We have to set **api_url = "nlst"** in our functions for this to work, but otherwise the steps are the same.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(modality = "CT",
                      manufacturer = "Philips",
                      api_url = "nlst")

print(len(data), 'Series returned')

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 2, api_url = "nlst", format = "df")
display(df)

## 4.3 Download a "shared cart"
It's possible to use https://nbia.cancerimagingarchive.net to create a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" which includes a specific set of scans you'd like to share with others.

After creating a Shared Cart you will receive a URL that looks like https://nbia.cancerimagingarchive.net/nbia-search/?saved-cart=nbia-49121659384603347 which can be shared with others.  Try clicking the link to see what this looks like on the TCIA website.  Then use the code below to see how you can use the cart name at the end of the URL to download the related scans via the API.

In [ ]:
# getSharedCart metadata
data = nbia.getSharedCart(name = "nbia-49121659384603347")
print(len(data), 'Series returned')

We'll skip the use of the **number** parameter this time since the full cart is only 4 series.  Let's also try **format = "csv"** to save a spreadsheet of the metadata in addition to returning a dataframe.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, format = "csv")
display(df)

## 4.4 Download data from a TCIA manifest file

When working with manifest files you can install the NBIA Data Retriever to open the manifest and download the data as shown in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Linux_Data_Retriever_App.ipynb).  However, there may be cases where you don't have administrative rights to install software or prefer using the REST API to download a manifest.  

In order to demonstrate this use case, let's assume that after you [Browse Collections](https://www.cancerimagingarchive.net/collections) you are interested in the [RIDER Breast MRI](https://doi.org/10.7937/K9/TCIA.2015.H1SXNUXL) collection.  We can find the URL of the manifest to download the full collection by looking at the blue "Download" button on that page.  Then we can download the manifest with the following commands.  

In [ ]:
# download manifest file from RIDER Breast MRI page
manifest = requests.get("https://www.cancerimagingarchive.net/wp-content/uploads/doiJNLP-Fo0H1NtD.tcia")
with open('RIDER_Breast_MRI.tcia', 'wb') as f:
    f.write(manifest.content)

If you open this manifest file in a text editor you'll notice that it contains several lines of download parameters that precede a list of Series Instance UIDs to download.  If we set **input_type = "manifest"** we can provide the path/filename to **downloadSeries()** and it will extract the UIDs from the file and download them.

In [ ]:
df = nbia.downloadSeries("RIDER_Breast_MRI.tcia", input_type = "manifest", number = 2, format = "df")
display(df)

## 4.5 Download data from a restricted collection
In some cases, you must specifically request access to collections before you can download them.  These are listed as **limited access** on the [Browse Collections](https://www.cancerimagingarchive.net/collections/) page. The steps to request access may vary depending on the collection, but will always require that you first [create a TCIA user account](https://wiki.cancerimagingarchive.net/x/xgHDAg). Once you've created an account, you can use your login/password to create an API token with the **getToken()** function from **tcia_utils** to verify your permissions. Tokens are valid for 2 hours and must be refreshed after that point, but **tcia_utils** monitors the timeout for you and automatically refreshes tokens if needed.

**Note:** Historically the **api_url** parameter needed to be specified if you wanted to access 'restricted' datasets in most functions.  As of **tcia_utils version 2.3** this is no longer needed.  Simply create a token with your credentials and you're good to go!

In [ ]:
nbia.getToken()

Let's say that we're interested in the [QIN-Breast-02](https://doi.org/10.7937/TCIA.2019.4cfm06rr) collection. As you can see on the collection page, you must email help@cancerimagingarchive.net to request access to the data. Once you've recieved approval we can use **nbia.getSeries()** to get a full list of series UIDs in this restricted collection.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(collection = "QIN-Breast-02")

if data != None:
    print(len(data), 'Series returned')
else:
    print('No data returned.')
    print('Did you forget to log in with your TCIA account in the previous cell?')
    print('Did you forget to obtain permission from the helpdesk to access this collection?')

Now we can download those scans just like before.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 2, format = "df")
display(df)

## 4.6 Download individual images from a series

It is also possible to download specific images if you don't want the entire series. Let's look at an example using the [**CMB-GEC**](https://doi.org/10.7937/E7KH-R486) collection.  First we'll get a list of the metadata about the series.

In [ ]:
nbia.getSeries(collection = "CMB-GEC")

Once we identify a series of interest we can pass its Series Instance UID to **nbia.getSopInstanceUids()** to obtain a list of SOP Instance UIDs for the individual images that are part of this series.

In [ ]:
seriesUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.919741553251398079475267746505"

nbia.getSopInstanceUids(seriesUid)

After we have both a series UID and SOP Instance UID, we can call the **nbia.downloadImage()** function to download a specific image from the series.

In [ ]:
sopUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.211684247543622814130853101548"

nbia.downloadImage(seriesUID, sopUID)

# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/).  It is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/) and hosted by the [University of Arkansas for Medical Sciences (UAMS)](https://www.uams.edu/)

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/). If you leverage this notebook or any TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7